In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [2]:
df = pd.read_csv('training.csv')

In [3]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [4]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [5]:
#BINNING NUMERICAL VALUES
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False)
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['MMR_factor_bin'] = pd.cut(df.MMR_factor.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [6]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Model','Trim','Color','Transmission','WheelType','Nationality','Size',
            'TopThreeAmericanName','VNZIP1','VNST','VehBCost_bin', 'VehOdo_bin', 'MMR_factor_bin', 'WarrantyCost_bin']]

In [7]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                    0
VehicleAge                 0
Make                       0
Model                      0
Trim                    1911
Color                      7
Transmission               8
WheelType               2577
Nationality                4
Size                       4
TopThreeAmericanName       4
VNZIP1                     0
VNST                       0
VehBCost_bin               0
VehOdo_bin                 0
MMR_factor_bin             0
WarrantyCost_bin           0
dtype: int64

In [8]:
df_new.dropna(subset = ['Transmission','Color','Nationality','Size','TopThreeAmericanName'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [11]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [12]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [13]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [14]:
df_new.Color.isnull().sum()
df_new.Transmission.isnull().sum()
df_new.Nationality.isnull().sum()
df_new.Size.isnull().sum()
df_new.TopThreeAmericanName.isnull().sum()
df_new.Trim.isnull().sum()
df_new.WheelType.isnull().sum()

0

In [15]:
df_new.isnull().sum()

Auction                 0
VehicleAge              0
Make                    0
Model                   0
Trim                    0
Color                   0
Transmission            0
WheelType               0
Nationality             0
Size                    0
TopThreeAmericanName    0
VNZIP1                  0
VNST                    0
VehBCost_bin            0
VehOdo_bin              0
MMR_factor_bin          0
WarrantyCost_bin        0
dtype: int64

In [17]:
#START PATTERN MINING
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df_new.head()

,Auction,VehicleAge,Make,Model,Trim,Color,Transmission,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,VehBCost_bin,VehOdo_bin,MMR_factor_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,SPECTRA_Model,EX_Trim,BLUE_Color,AUTO_Transmission,Covers_WheelType,OTHER ASIAN_Nationality,MEDIUM_Size,OTHER_TopThreeAmericanName,33619_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[60271.0, 71360.2)_VehOdo_bin","[7598.4, 11397.6)_MMR_factor_bin","[462.0, 1165.6)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,STRATUS V6 2.7L V6 M_Model,SXT_Trim,SILVER_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,32124_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[82449.4, 93538.6)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,CALIBER_Model,SE_Trim,BLACK_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,74135_VNZIP1,OK_VNST,"[7297.8, 10946.2)_VehBCost_bin","[38092.6, 49181.8)_VehOdo_bin","[7598.4, 11397.6)_MMR_factor_bin","[462.0, 1165.6)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,FREESTAR FWD V6 3.9L_Model,S_Trim,BROWN_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,VAN_Size,FORD_TopThreeAmericanName,85018_VNZIP1,AZ_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,TOWN & COUNTRY FWD V_Model,Bas_Trim,BLUE_Color,AUTO_Transmission,Alloy_WheelType,AMERICAN_Nationality,VAN_Size,CHRYSLER_TopThreeAmericanName,75236_VNZIP1,TX_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"


In [22]:
baskets = df_new.values.tolist()

In [25]:
baskets

[['ADESA_Auction',
  '2_VehicleAge',
  'KIA_Make',
  'SPECTRA_Model',
  'EX_Trim',
  'BLUE_Color',
  'AUTO_Transmission',
  'Covers_WheelType',
  'OTHER ASIAN_Nationality',
  'MEDIUM_Size',
  'OTHER_TopThreeAmericanName',
  '33619_VNZIP1',
  'FL_VNST',
  '[3649.4, 7297.8)_VehBCost_bin',
  '[60271.0, 71360.2)_VehOdo_bin',
  '[7598.4, 11397.6)_MMR_factor_bin',
  '[462.0, 1165.6)_WarrantyCost_bin'],
 ['MANHEIM_Auction',
  '3_VehicleAge',
  'DODGE_Make',
  'STRATUS V6 2.7L V6 M_Model',
  'SXT_Trim',
  'SILVER_Color',
  'AUTO_Transmission',
  'Covers_WheelType',
  'AMERICAN_Nationality',
  'MEDIUM_Size',
  'CHRYSLER_TopThreeAmericanName',
  '32124_VNZIP1',
  'FL_VNST',
  '[3649.4, 7297.8)_VehBCost_bin',
  '[82449.4, 93538.6)_VehOdo_bin',
  '[3799.2, 7598.4)_MMR_factor_bin',
  '[1165.6, 1869.2)_WarrantyCost_bin'],
 ['OTHER_Auction',
  '2_VehicleAge',
  'DODGE_Make',
  'CALIBER_Model',
  'SE_Trim',
  'BLACK_Color',
  'AUTO_Transmission',
  'Covers_WheelType',
  'AMERICAN_Nationality',
  'MEDI

In [28]:
from fim import apriori

In [30]:
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [29]:
rules = apriori(baskets, supp=20, zmin=2, target='r', conf=70, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

209
0 -- ('AMERICAN_Nationality', ('SILVER_Color',), 83.58515777572207, 20.233323054784663, 0.9922770352181137, 82.93963254593176)
1 -- ('AUTO_Transmission', ('SILVER_Color',), 96.49158872100593, 20.233323054784663, 1.0005596795627067, 96.54559309118618)
2 -- ('AMERICAN_Nationality', ('Bas_Trim',), 83.58515777572207, 20.57594134374893, 0.9695785708485045, 81.04237782033137)
3 -- ('AUTO_Transmission', ('Bas_Trim',), 96.49158872100593, 20.57594134374893, 0.9892485788534935, 95.4541670135709)
4 -- ('AMERICAN_Nationality', ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission'), 83.58515777572207, 20.27957652379484, 1.0436783505509324, 87.23601959790506)
5 -- ('AMERICAN_Nationality', ('[82449.4, 93538.6)_VehOdo_bin',), 83.58515777572207, 20.86374070647891, 1.03665713485919, 86.64915017653337)
6 -- ('AUTO_Transmission', ('[82449.4, 93538.6)_VehOdo_bin',), 96.49158872100593, 20.86374070647891, 1.0073427105786354, 97.200098530257)
7 -- ('AMERICAN_Nationality', ('3_VehicleAge', 'AUTO_Transmissi